In [13]:
!nvidia-smi
!git clone https://github.com/taqu/txt2mesh/tree/initial .
!git clone --recursive https://github.com/threestudio-project/threestudio.git

Sun Jun  4 07:17:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
!pip install ninja
!pip install lightning==2.0.0 omegaconf==2.3.0 jaxtyping typeguard diffusers transformers accelerate opencv-python tensorboard matplotlib imageio imageio[ffmpeg] trimesh bitsandbytes sentencepiece safetensors huggingface_hub libigl xatlas datasets 
!pip install open3d plotly # mesh visualization
!pip install git+https://github.com/KAIR-BAIR/nerfacc.git@v0.5.2
!pip install git+https://github.com/NVlabs/nvdiffrast.git
!pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/KAIR-BAIR/nerfacc.git (to revision v0.5.2) to /tmp/pip-req-build-jnev8jri
  Running command git clone --filter=blob:none --quiet https://github.com/KAIR-BAIR/nerfacc.git /tmp/pip-req-build-jnev8jri
  Running command git checkout -q d84cdf3afd7dcfc42150e0f0506db58a5ce62812
  Resolved https://github.com/KAIR-BAIR/nerfacc.git to commit d84cdf3afd7dcfc42150e0f0506db58a5ce62812
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple

In [55]:
import os
import shutil
import datetime
import glob

def save_last_train_count(path:str)->None:
    src_path = '/content/train_count.txt'
    if False == os.path.exists(src_path):
        return
    date = datetime.datetime.utcnow()
    dst_path = os.path.join(path, 'train_count'+date.strftime('%Y%m%d%H%M')+'.txt')
    shutil.copy(src_path, dst_path)

def restore_last_train_count(path:str)->None:
    try:
        file_path = os.path.join(path, 'train_count*.txt')
        files = glob.glob(file_path)
        if None == files:
            return
        files.sort()
        file_path = os.path.join(path, files[-1])
        print(file_path)
        shutil.copy(file_path, '/content/train_count.txt')
    except Exception as e:
        print(e)
    
restore_last_train_count('/content/drive/MyDrive/Sheep/ckpts') 
#save_last_train_count('/content/drive/MyDrive/Sheep/ckpts')

/content/drive/MyDrive/Sheep/ckpts/train_count202306041938.txt


In [56]:
import torch
import datasets
import sys
import os
import secrets
import asyncio
import shutil
import datetime
import glob

def load_count(filename):
    count = 0
    try:
        with open(filename, mode='r') as f:
            for l in f.readlines():
                try:
                    count = int(l)
                    break
                except:
                    continue
    except:
        pass
    finally:
        return count

def save_count(filename, count):
    try:
        with open(filename, mode='w') as f:
            f.write(str(count))
    except:
        pass
    finally:
        return

async def run_command(command: str, args: list[str]) -> None:
    process = await asyncio.create_subprocess_exec(
        command,
        *args,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE)
    while True:
        if process.stdout.at_eof() and process.stderr.at_eof():
            break;
        stdout = (await process.stdout.readline()).decode()
        if stdout:
            print(stdout, end='', flush=True)
        stderr = (await process.stderr.readline()).decode()
        if stderr:
            print(stderr, end='', flush=True, file=sys.stderr)
        await asyncio.sleep(1)
    await process.communicate()

def find_last_checkpoint(path:str) -> str:
    files = os.listdir(path)
    dirs = [f for f in files if os.path.isdir(os.path.join(path,f))]
    dirs.sort()
    if len(dirs)<=0:
        return None
    check_point = os.path.join(path, dirs[-1], 'ckpts', 'last.ckpt')
    return check_point if os.path.exists(check_point) else None

def save_last_checkpoint(path:str) -> None:
    check_point = find_last_checkpoint('/content/threestudio/outputs/sheep')
    if None == check_point:
        return
    dst_dir = '/content/drive/MyDrive/Sheep/ckpts/'
    date = datetime.datetime.utcnow()
    dst_path = os.path.join(dst_dir, 'ckpt'+date.strftime('%Y%m%d%H%M')+'.ckpt')
    shutil.copy(check_point, dst_path)

def restore_last_checkpoint(path:str)->None:
    try:
        file_path = os.path.join(path, 'ckpt*.ckpt')
        files = glob.glob(file_path)
        if None == files:
            return
        files.sort()
        file_path = os.path.join(path, files[-1])
        print(file_path)
        shutil.copy(file_path, '/content/last.ckpt')
    except Exception as e:
        print(e)

def save_last_train_count(path:str)->None:
    src_path = '/content/train_count.txt'
    if False == os.path.exists(src_path):
        return
    date = datetime.datetime.utcnow()
    dst_path = os.path.join(path, 'train_count'+date.strftime('%Y%m%d%H%M')+'.txt')
    shutil.copy(src_path, dst_path)

def restore_last_train_count(path:str)->None:
    try:
        file_path = os.path.join(path, 'train_count*.txt')
        files = glob.glob(file_path)
        if None == files:
            return
        files.sort()
        file_path = os.path.join(path, files[-1])
        print(file_path)
        shutil.copy(file_path, '/content/train_count.txt')
    except Exception as e:
        print(e)

prompts = []
dataset = datasets.load_dataset("FredZhang7/stable-diffusion-prompts-2.47M")
prompts = prompts + dataset['train']['text']

dataset = datasets.load_dataset('Gustavosta/Stable-Diffusion-Prompts')
prompts = prompts + dataset['train']['Prompt']

#dataset = datasets.load_dataset("FredZhang7/anime-prompts-180K")
#prompts = prompts + dataset['train']['safebooru_clean']
#prompts = prompts + dataset['train']['danbooru_clean']
#prompts = prompts + dataset['train']['danbooru_raw']                         

dataset = None
restore_last_checkpoint('/content/drive/MyDrive/Sheep/ckpts')
restore_last_train_count('/content/drive/MyDrive/Sheep/ckpts') 
count_path = '/content/train_count.txt'
train_count = load_count(count_path)
print(min(len(prompts), train_count))
max_train_count = max(len(prompts), train_count)
print(max_train_count)
conf_file = os.path.join(os.getcwd(), 'threestudio/configs/prolificdreamer.yaml')

os.chdir('./threestudio')
torch.set_float32_matmul_precision('medium')
base_config = ['launch.py','--config', conf_file, '--train', '--gpu', '0', 'name=\"sheep\"','tag=\"sheep\"', 'data.width=512', 'data.height=512']
#base_config = ['launch.py','--config', conf_file, '--train', '--gpu', '0', 'name=\"sheep\"','tag=\"sheep\"','data.width=1', 'data.height=1']
extra_condig = ['trainer.max_steps=10000', 'system.guidance.token_merging=true', 'system.guidance.enable_attention_slicing=true']
#extra_condig = ['trainer.max_steps=1', 'trainer.max_epochs=1', 'trainer.fast_dev_run=True', 'data.batch_size=1', 'system.guidance.token_merging=true', 'system.guidance.enable_attention_slicing=true']
for i in range(train_count, 1):
    seed = secrets.randbits(64)
    args = base_config + ['seed='+str(seed)] + ['system.prompt_processor.prompt=' + prompts[i]]
    try:
        print(args)
        await run_command('python3', args)
    except Exception as e:
        print(e)
        break
    train_count += 1
    save_count(count_path, train_count)
    save_last_checkpoint('/content/drive/MyDrive/Sheep/ckpts')
    save_last_train_count('/content/drive/MyDrive/Sheep/ckpts')
os.chdir('..')


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

/content/drive/MyDrive/Sheep/ckpts/ckpt202306041846.ckpt
/content/drive/MyDrive/Sheep/ckpts/train_count202306041938.txt
1
2546740
